In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
# Load the datasets
earthquake_data_path = 'earthquakesIceland.csv' 
eruption_data_path = 'Eruption_Results.csv'      

earthquake_data = pd.read_csv(earthquake_data_path)
eruption_data = pd.read_csv(eruption_data_path, encoding='ISO-8859-1')

# Setting the first row as the header for the eruption data
eruption_data.columns = eruption_data.iloc[0]
eruption_data = eruption_data[1:]

# Convert 'Start Year' and 'End Year' in eruption data to numeric
eruption_data['Start Year'] = pd.to_numeric(eruption_data['Start Year'], errors='coerce')
eruption_data['End Year'] = pd.to_numeric(eruption_data['End Year'], errors='coerce')

# Drop rows with NaN in 'Start Year' or 'End Year'
eruption_data.dropna(subset=['Start Year', 'End Year'], inplace=True)

print("Datasets loaded and cleaned.")

# Define if the seismic activity is followed by an eruption
earthquake_data['Year'] = pd.to_datetime(earthquake_data['time']).dt.year
earthquake_data['During Eruption'] = 0

for _, eruption_row in eruption_data.iterrows():
    eruption_year_range = range(int(eruption_row['Start Year']), int(eruption_row['End Year']) + 1)
    matched = earthquake_data['Year'].isin(eruption_year_range).sum()
    if matched > 0:
        earthquake_data.loc[earthquake_data['Year'].isin(eruption_year_range), 'During Eruption'] = 1

print("Data integration completed.")



print("Script completed.")


Datasets loaded and cleaned.
Data integration completed.
Script completed.


In [3]:
# Check if there are enough classes for model training
if earthquake_data['During Eruption'].nunique() < 2:
    print("Not enough classes for model training. Exiting.")
else:
    # Feature Selection and Model Building
    X = earthquake_data[['latitude', 'longitude', 'depth', 'mag']]
    y = earthquake_data['During Eruption']

    # Splitting the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Logistic Regression Model
    model = LogisticRegression()

    # Training the model
    model.fit(X_train, y_train)

    print("Model training completed.")

    # Making predictions
    predictions = model.predict(X_test)

    # Model Evaluation
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("\nClassification Report:")
    print(classification_report(y_test, predictions))

Model training completed.
Confusion Matrix:
[[ 20  60]
 [ 11 217]]

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.25      0.36        80
           1       0.78      0.95      0.86       228

    accuracy                           0.77       308
   macro avg       0.71      0.60      0.61       308
weighted avg       0.75      0.77      0.73       308



In [4]:
print(X_train.head())

     latitude  longitude  depth  mag
227   64.6366   -17.7710   7.30  4.6
971   64.5070   -17.6250  33.00  5.2
497   64.4457   -17.6238   7.11  5.0
919   66.2680   -17.1240  10.00  4.2
177   66.2493   -18.6779  10.00  5.4


In [5]:

# Splitting the dataset
X = earthquake_data[['latitude', 'longitude', 'depth', 'mag']]
y = earthquake_data['During Eruption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM": SVC(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier()
}

# Training and evaluating models
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{name} Accuracy: {accuracy:.4f}")


Logistic Regression Accuracy: 0.7695
Decision Tree Accuracy: 0.7857
Random Forest Accuracy: 0.8214
Gradient Boosting Accuracy: 0.8442
SVM Accuracy: 0.7403
XGBoost Accuracy: 0.8377
[LightGBM] [Info] Number of positive: 516, number of negative: 200
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000122 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 589
[LightGBM] [Info] Number of data points in the train set: 716, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.720670 -> initscore=0.947789
[LightGBM] [Info] Start training from score 0.947789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report


# Splitting the dataset
X = earthquake_data[['latitude', 'longitude', 'depth', 'mag']]
y = earthquake_data['During Eruption']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Gradient Boosting Classifier
gb_model = GradientBoostingClassifier()

# Hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0]
}

# GridSearchCV
grid_search = GridSearchCV(gb_model, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best parameters from GridSearchCV:")
print(grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# RandomizedSearchCV
random_search = RandomizedSearchCV(gb_model, param_grid, n_iter=10, cv=3, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

print("\nBest parameters from RandomizedSearchCV:")
print(random_search.best_params_)
print("Best score:", random_search.best_score_)

# Evaluating the best model from GridSearchCV
best_model_grid = grid_search.best_estimator_
grid_predictions = best_model_grid.predict(X_test)
print("\nClassification report for the best model from GridSearchCV:")
print(classification_report(y_test, grid_predictions))

# Evaluating the best model from RandomizedSearchCV
best_model_random = random_search.best_estimator_
random_predictions = best_model_random.predict(X_test)
print("\nClassification report for the best model from RandomizedSearchCV:")
print(classification_report(y_test, random_predictions))


Best parameters from GridSearchCV:
{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300, 'subsample': 0.8}
Best score: 0.8044606964124563

Best parameters from RandomizedSearchCV:
{'subsample': 0.8, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1}
Best score: 0.8016771562181358

Classification report for the best model from GridSearchCV:
              precision    recall  f1-score   support

           0       0.75      0.59      0.66        80
           1       0.87      0.93      0.90       228

    accuracy                           0.84       308
   macro avg       0.81      0.76      0.78       308
weighted avg       0.83      0.84      0.83       308


Classification report for the best model from RandomizedSearchCV:
              precision    recall  f1-score   support

           0       0.74      0.61      0.67        80
           1       0.87      0.93      0.90       228

    accuracy                           0.84       308
   macro avg       0.81      0.

#### I tested both GridSearchCV and RandomizedSearchCV to determine the best parameters. Because the daily seismic activity is not very high, I chose to use GridSearchCV, but I kept both in case the number of events increases, and GridSearchCV becomes computationally expensive.


In [7]:
import pickle

best_model = grid_search.best_estimator_

# Save the model to a file
with open('best_gradient_boosting_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

print("Model saved successfully.")


Model saved successfully.
